In [63]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

#filename = "math sample for API testing.csv"
#filename = "Big Math Subset.csv"
filename = "Math Test.csv"

In [140]:
def process_file(filename, tagging_df):
    parsed_data = []
    error_data = []
    component_holding = []
    initial_components = True
    content_focus_component_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #for i in range(1):
        #    l = reader.next()    #########See if I can simplify this

        for line in reader:
            error_test = False
            component_list = line['Component'].split("; ")
            content_focus_list = line['Content Focus'].split("; ")
            #first pass, only process those items that have one component or only one content focus, build data frame of 
            #allowable strand/substrand/content focus/component combinations
            if len(component_list) == 1 or len(content_focus_list) == 1:
                print "hi hi!"
                row, error_test, component_holding, content_focus_component_df = parse_line( \
                    line, tagging_df, content_focus_list, component_list, error_test, component_holding, \
                    content_focus_component_df, initial_components)
                if error_test == True:
                    error_data.append(row)                
                else: parsed_data.append(row)
                if component_holding == True:
                    component_holding.append(line)
            else: component_holding.append(line)
    initial_components = False
    #process items sent to component_holding
    return parsed_data, error_data, component_holding, content_focus_component_df

In [132]:
def parse_line(line, tagging_df, content_focus_list, component_list, error_test, component_holding, \
               content_focus_component_df, initial_components):
    row = {}
    strand_dict = {}
    strand_list = []
    differentiator_list = []
    content_focus_component_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
    InternalID = int(line['Internal ID'])
    Strand = line['Strand'].split("; ")
    for strand in Strand:
        strand_sub_dict = {}
        content_focus_df = tagging_df['Content_Focus']
        GUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
        strand_sub_dict = {'Strand': strand, 'GUID' : GUID, 'Substrand': []}
        strand_list.append(strand_sub_dict)
    if error_test == False:
        substrand_list = line['Sub-Strand'].split("; ")
        strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)
        if error_test == False:
            strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, \
                                                            error_test)
            print error_test
            print initial_components
            if error_test == False and initial_components == True:
                print "Hey, baby"
                component_hold = False
                strand_list, error_test, component_hold, content_focus_component_df = processInitialComponent(\
                    strand_list, substrand_list, content_focus_list, component_list, tagging_df['Component'], error_test, \
                    component_hold, content_focus_component_df)
            #else:  
                ###### Insert 2nd processing of component    
            if error_test == False:
                differentiator_keys = ['Shape', 'Units', 'Steps']            
                for differentiator in differentiator_keys:
                    differentiator_sub_list = []
                    differentiator_df = tagging_df[differentiator]
                    differentiator_values = line[differentiator].split("; ")
                    for value in differentiator_values:
                        differentiator_sub_dict = {}
                        GUID, error_test = get_GUID_lite(value, differentiator_df, error_test)
                        differentiator_sub_dict = {differentiator: value, 'GUID': GUID}
                        differentiator_sub_list.append(differentiator_sub_dict)
                        if error_test == True:
                            break
                    differentiator_list.append(differentiator_sub_list)
    row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": differentiator_list}
    return row, error_test, component_holding, content_focus_component_df

In [135]:
data = []
parsed_data, error_data, component_holding, content_focus_component_df = process_file(filename, tagging_dict)
#pprint.pprint(bad_data)

hi hi!
False
True
Hey, baby
First Quadrant
hello


In [136]:
len(error_data)
pprint.pprint(error_data)

[]


In [137]:
#pprint.pprint(bad_data)
print len(component_holding)
print len(parsed_data)

0
1


In [138]:
print content_focus_component_df

Empty DataFrame
Columns: [Strand, Sub-Strand, Content Focus, Component, ComponentGUID]
Index: []


In [139]:
pprint.pprint(parsed_data)

[{'Differentiators': [[{'GUID': '1E24B42A-97C8-11E6-8048-1FDE29C466BA',
                        'Shape': 'Point'}],
                      [{'GUID': '7C0B3838-97C4-11E6-B4CA-B3D929C466BA',
                        'Units': 'Generic'}],
                      [{'GUID': 'N/A', 'Steps': 'N/A'}]],
  'Internal_ID': 329166,
  'Organizers': [{'GUID': 'D66D958E-8BFB-11E6-921B-89CC29C466BA',
                  'Strand': 'Geometry and Measurement',
                  'Substrand': [{'Content Focus': [{'Component': [{'Component': 'First Quadrant',
                                                                   'GUID': '342BC6C6-9D7E-11E6-BB08-D0B929C466BA'}],
                                                    'Content Focus': 'Graph points',
                                                    'GUID': 'CAA6BC40-8C1D-11E6-867F-EA7829C466BA'}],
                                 'GUID': '1573E894-8C17-11E6-81DC-467029C466BA',
                                 'Substrand': 'Coordinate Plane'}]}]}]


In [124]:
df = component_df
strand = 'Geometry and Measurement'
substrand = 'Coordinate Plane'
component = 'First Quadrant'
df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component))]

#print df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(component))]

,Asset Subject,Example Content Focus,Strand,Sub-Strand,Subgroup,Title,Unique ID
37,Mathematics,graph points,geometry and measurement,coordinate plane,Graphing and Plotting,first quadrant,342BC6C6-9D7E-11E6-BB08-D0B929C466BA


In [134]:
def processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, component_hold, \
                            content_focus_component_df):
    GUID = None
    for component in component_list:
        if len(df[(df['Title']==lowercase(component))]) == 0:  
            error_test = True
            error_message = 'Error: Component ' + component + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, \
                                                                                    'GUID': error_message})
            break
    if len(content_focus_list) == 1:
        for component in component_list:
            print component
            strand_list, error_test, component_hold, guid, content_focus_component_df = getComponentGUID( \
                strand_list[0]['Strand'], strand_list, substrand_list[0], content_focus_list[0], component, df, error_test, \
                component_hold, content_focus_component_df) 
            #add to content_focus_component_df
    elif len(substrand_list) == len(component_list):
        strand_list, error_test, component_hold, content_focus_component_df = matchInitialComponent(strand_list, substrand_list,\ 
                            content_focus_list, component_list, df, error_test, len(component_list), content_focus_component_df)            
    else: 
        component_hold = True
    return strand_list, error_test, component_hold, content_focus_component_df

def getComponentGUID(strand, strand_list, substrand, content_focus, component, df, error_test, component_hold, \
                     content_focus_component_df): 
    ComponentGUID = ""
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate( \
            strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    content_focus_index = next(index for (index, d) in enumerate( \
            strand_list[strand_index]['Substrand'][substrand_index]['Content Focus']) if d["Content Focus"] == content_focus)
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
              (df['Title']==lowercase(component))]) == 1:
        print "hello"
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                              (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component))]) == 1:
        print "hi"
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                              (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & 
                (df['Title']==lowercase(component))]) > 1:
        print "wasssssuuuuuup"
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'Holding'})
        component_hold = True
    else: 
        print "head desk"
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'Error: No unique Strand/Substrand/Component match found in master list'})
        error_test = True
    return strand_list, error_test, component_hold, ComponentGUID, content_focus_component_df

def matchInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, matchlimit, \
                          content_focus_component_df):
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + '; ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=component_list)
    strand_substrand_names[0].split('; ')[0]

    for strand_substrand in strand_substrand_names:
        #Need to use substrand and also pull strand from strand list, might be easier to build from strand, down
        for component in component_list:
            initial_match_length = len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                                          (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1]))& \
                                          (df['Title']==lowercase(component))])
            if  initial_match_length == 1:
                match_status.ix[strand_substrand, component] = True
            elif initial_match_length > 1:
                    if len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                              (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1])) & \
                              (df['Example Content Focus']==lowercase(content_focus_list[0])) & \
                              (df['Title']==lowercase(component))]) == 1:
                        match_status.ix[strand_substrand, component] = True
                
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test, GUID = getComponentGUID(pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1], \
                                                             content_focus_list[0], pair[1], component_df, \
                                                             error_test, component_hold)
            if len(content_focus_component_df[(content_focus_component_df['Strand']==lowercase(pair[0].split('; ')[0])) & \
                                              (content_focus_component_df['Sub-Strand']==lowercase( pair[0].split('; ')[1])) & \
                                              (content_focus_component_df['Content Focus']==lowercase(content_focus_list[0])) & \
                                              (content_focus_component_df['Component']==lowercase(pair[1]))]) == 0:
                #add group to content_focus_component_df
                df_add = {'Strand': lowercase(pair[0].split('; ')[0]), 'Sub-Strand': lowercase(pair[0].split('; ')[1]), \
                          'Content Focus': lowercase(content_focus_list[0]), "Component": lowercase(pair[1]), \
                          "ComponentGUID": "test"}
                content_focus_component_df = content_focus_component_df.append(df_add, ignore_index = True)

    else: 
        strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'GUID': 'Error: Aha!'})
        error_test = True
    return strand_list, error_test, component_hold, content_focus_component_df

In [99]:
def processSubstrands(strand_list, substrand_list, df, error_test):
    GUID = None
    for substrand in substrand_list:
        if len(df[(df['Title']==lowercase(substrand))]) == 0:  
            error_test = True
            error_message = 'Error: Substrand ' + substrand + ' not found in master list'
            strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': error_message})
            break
    if len(substrand_list) < len(strand_list):
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': 'Error: more strands than substrands'})
    elif len(strand_list) == 1: 
        for substrand in substrand_list:
            strand_list, error_test = getSubstrandGUID(strand_list[0]['Strand'], strand_list, substrand, df, error_test)    
    elif len(substrand_list) == 2 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 2)
    elif len(substrand_list) == 3 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 3)
    else: 
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': \ 
                                            'Error: Multiple substrands, cannot parse yet'})
    return strand_list, error_test

#Build a data frame to track match status to determine whether strand-substrand pairings can be deduced    
def matchStrandsSubstrands(strand_list, substrand_list, df, error_test, matchlimit):
    strand_names = []
    for strand in strand_list:
        strand_names.append(strand['Strand'])

    match_status = pd.DataFrame(False,index=strand_names,columns=substrand_list)

    for strand in strand_names:
        for substrand in substrand_list:
            if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
                match_status.ix[strand,substrand] = True
    
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        for pair in true_pairs:
            strand_list, error_test = getSubstrandGUID(pair[0], strand_list, pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], \ 
                                            'GUID': 'Error: Ambiguous strand/substrand relationships'})
        error_test = True
    return strand_list, error_test

def getSubstrandGUID(strand, strand_list, substrand, df, error_test): 
    if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
        SubstrandGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand)), 'Unique ID'].item()
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append( \
            {'Substrand': substrand, 'GUID': SubstrandGUID, 'Content Focus': []})
    else: 
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append( \
            {'Substrand': substrand, 'GUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

    

In [145]:
def processContentFocus(strand_list, substrand_list, content_focus_list, df, error_test):
    GUID = None
    for content_focus in content_focus_list:
        if len(df[(df['Title']==lowercase(content_focus))]) == 0:  
            error_test = True
            error_message = 'Error: Content Focus ' + content_focus + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': error_message})
            break
    if len(content_focus_list) < len(substrand_list):
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                                'Error: more Substrands than Content Focus terms'})
    elif len(substrand_list) == 1: 
        for content_focus in content_focus_list:
            strand_list, error_test = getContentFocusGUID(strand_list[0]['Strand'], strand_list, substrand_list[0], \
                                                          content_focus, df, error_test)    
    elif len(substrand_list) == len(content_focus_list):
        strand_list, error_test = matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, \
                                                              len(content_focus_list))
    else: 
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                                'Error: Multiple content Focus terms, cannot parse yet'})
    return strand_list, error_test

def matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, matchlimit):
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + '; ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=content_focus_list)
    for strand_substrand in strand_substrand_names:
        for content_focus in content_focus_list:
            #print strand_substrand
            #print content_focus
            if len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                      (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1]))& \
                      (df['Title']==lowercase(content_focus))]) == 1:
                match_status.ix[strand_substrand, content_focus] = True

    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test = getContentFocusGUID(pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1],  \
                                                          pair[1], df, error_test)
    elif int(match_status.sum().sum()) < matchlimit:
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                                'Error: One or more content focus terms not found'})
        error_test = True
    else: 
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                            'Error: Ambiguous strand/substrand/content focus relationships'})
        error_test = True
    return strand_list, error_test

def getContentFocusGUID(strand, strand_list, substrand, content_focus, df, error_test): 
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate( \
                                                     strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
              (df['Title']==lowercase(content_focus))]) == 1:
        ContentFocusGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                                 (df['Title']==lowercase(content_focus)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append( \
                            {'Content Focus': content_focus, 'GUID': ContentFocusGUID, 'Component': []})
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append( \
                            {'Content Focus': content_focus, 'GUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

In [101]:
strands = ['Numbers and Operations']
substrand_list = ['Operations']
content_focus_list = ['Calculate']
component_list = ['Addition']
error_test = False
component_hold = False
#strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)
content_focus_component_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])

strand_list=[]

for strand in strands:
    strand_sub_dict = {}
    #content_focus_df = tagging_df['Content_Focus']
    StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
    strand_sub_dict = {'Strand': strand, 'GUID' : StrandGUID, 'Substrand': []}
    strand_list.append(strand_sub_dict)
                        
strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)

strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)
pprint.pprint(strand_list)
strand_list, error_test, component_hold, content_focus_component_df = processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, component_df, error_test, component_hold, content_focus_component_df)

pprint.pprint(strand_list[0])

[{'GUID': 'D66EB2C0-8BFB-11E6-921B-89CC29C466BA',
  'Strand': 'Numbers and Operations',
  'Substrand': [{'Content Focus': [{'Component': [],
                                    'Content Focus': 'Calculate',
                                    'GUID': 'CBFF09D0-8C1D-11E6-867F-EA7829C466BA'}],
                 'GUID': '168B2CF6-8C17-11E6-81DC-467029C466BA',
                 'Substrand': 'Operations'}]}]
Addition


TypeError: getComponentGUID() takes exactly 8 arguments (9 given)

In [14]:
def lowercase(text):
    return unicode(text, "latin2").lower()

In [122]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
        df_lowercase(data, 'Example Content Focus')
    return data

In [16]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [146]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, \
                'Content_Focus': content_focus_df, 'Component': component_df}

In [18]:
def get_GUID_lite(tag, df, error_test): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        else: 
            GUID = "Error: no matches found"
            error_test = True
    else: 
        GUID = "Error: resolve multiple values"
        error_test = True
    return GUID, error_test

In [19]:
number_test = "Square roots"
error_test = False
print get_GUID_lite(number_test, number_df, error_test)

('1690A96C-97C3-11E6-88DC-EDD729C466BA', False)


In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument

tag = 'N/A'
if tag == r'N/A':
    print tag


In [ ]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
                        {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
            {Units: [Units1, ...]}
            {Steps: Steps}
            {Context: Context}
            {SIG Stem: [Sig Stem1, ...]}
            {SIG Options: [SIG Options1, ...]}
            {Additional Modifiers: [AM1, ...]}
}

'''